In [1]:
# I'm importing the libraries and loading the dataset
from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical

# I'm training the images and testing the labels and setting it equal to mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#I'm defining the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

#Now I'm reshaping with the format [samples][width][height][channels]
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#I'm compiling the optimizer, loss, and metrics
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#I'm fitting the model with the train_images and train_labels
model.fit(train_images, train_labels, epochs=5, batch_size=64)

#I'm testing the loss and accuracy by making the model evaluate the tes_images and labels
test_loss, test_acc = model.evaluate(test_images, test_labels)

#I'm testing the accuracy
print(test_acc)

#I'm saving the model as mnist.h5
model.save('mnist.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [1]:
# I started by importing the libraries
from tkinter import *

import cv2
import numpy as np
from PIL import ImageGrab
from keras.models import load_model
import webbrowser

# I'm using Module 1 for the backend but the saved model is Module 2 for recognizing the digits
model = load_model('mnist.h5')
image_folder = "img/"

# I'm initializing by creating a window and giving the title Handwritten Digit Recognition
root = Tk()
root.resizable(0, 0)
root.title("Handwritten Digit Recognition")

lastx, lasty = None, None
image_number = 0

# I created the drawing area so users can draw the numbers
cv = Canvas(root, width=600, height=500, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=NSEW, columnspan=2)

# I'm clearing the drawing area when the users clicks clear output button
def clear_widget():
    global cv
    cv.delete('all')

# the draw and activate functions lets the user draw digits
def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event.y
    cv.create_line((lastx, lasty, x, y), width=8, fill='black', capstyle=ROUND, smooth=TRUE, splinesteps=12)
    lastx, lasty = x, y


def activate_event(event):
    global lastx, lasty
    cv.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event.y


cv.bind('<Button-1>', activate_event)

# the recognize function predicts the digits that the user draws on the drawing area
def Recognize_Digit():
    global image_number
    filename = f'img_{image_number}.png'
    widget = cv

    x = root.winfo_rootx() + widget.winfo_rootx()
    y = root.winfo_rooty() + widget.winfo_rooty()
    x1 = x + widget.winfo_width()
    y1 = y + widget.winfo_height()
    print(x, y, x1, y1)
    
    # I'm getting the image and saving it
    ImageGrab.grab().crop((x, y, x1, y1)).save(image_folder + filename)

    image = cv2.imread(image_folder + filename, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # I'm making a rectangle box around each curve
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)

        # Then cropping out the digit from the image to the contours in the for loop
        digit = th[y:y + h, x:x + w]

        # Then I'm resizing the digit to (18, 18)
        resized_digit = cv2.resize(digit, (18, 18))

        # and padding the digit with 5 pixels to produce the image
        padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)

        digit = padded_digit.reshape(1, 28, 28, 1)
        digit = digit / 255.0

        pred = model.predict([digit])[0]
        final_pred = np.argmax(pred)

        data = str(final_pred) + ' ' + str(int(max(pred) * 100)) + '%'

        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 1
        cv2.putText(image, data, (x, y - 5), font, fontScale, color, thickness)

    cv2.imshow('Predictions', image)
    cv2.waitKey(0)
    
# and then these are the buttons and the height, width, text and alignment of the buttons  
btn_save = Button(text='Recognize Digits',width=15, height=3, command=Recognize_Digit)
btn_save.grid(row=2, column=0, pady=1, padx=1)
button_clear = Button(text='Clear Output',width=15, height=3, command=clear_widget)
button_clear.grid(row=2, column=1, pady=1, padx=1)

root.mainloop()

172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
172 220 776 724
